In [65]:
import os
import io
import time
import json
import datetime as dt
import typing
import requests

import pandas as pd
import snowflake.connector
from pypardot.client import PardotAPI, PardotAPIError
from xml.etree import ElementTree
import snowflake.connector


In [66]:
PARDOT_EMAIL="segment_integrations@discoveryed.com"
PARDOT_USER_KEY="f5a1dc61d079e35d9a2066a4a8498c32"
PARDOT_SF_CONSUMER_KEY="3MVG9IHf89I1t8hpom1l0QzbTANHH.MOASIZ3yPPhu6hFI_uipXiYi7ku85yMWg_2gXxu5bzAyWCevOHp4jrf"
PARDOT_SF_CONSUMER_SECRET="BC6C2AA5D41DC0EBBBB3376CD329E92E66BD8D85DF85F9FAB9F22E86D857A29E"
PARDOT_SF_REFRESH_TOKEN="5Aep8618yVsldz6rZPMv4ouelrGoRNAfdaLSjo3.ILw4jdNGemJVI_MNYFoNkN9g39GUXqUEgzw3ylEVMjxAU3X"
PARDOT_BUSINESS_UNIT_ID="0Uv4P000000TNB0SAO"
PARDOT_API_VERSION=4
PARDOT_MAX_RESULT_COUNT = 200
SNOWFLAKE_ACCOUNT_IDENTIFIER="UK29315.us-east-1"
SNOWFLAKE_USER="PardotEtlUser@discoveryed.com"
SNOWFLAKE_PASS="p@rd0tEtlUser"

In [67]:
SET_DATA_TYPE_CREATED = {
    # This set is for the data types that do not have a "updated_after" filter,
    # and must be queried usng the "created_after" filter
    "EmailClick",
    "TagObject",
}


In [68]:
def get_client_pardot() -> PardotAPI:
    try:
        p = PardotAPI(
            email=PARDOT_EMAIL,
            user_key=PARDOT_USER_KEY,
            sf_consumer_key=PARDOT_SF_CONSUMER_KEY,
            sf_consumer_secret=PARDOT_SF_CONSUMER_SECRET,
            sf_refresh_token=PARDOT_SF_REFRESH_TOKEN,
            business_unit_id=PARDOT_BUSINESS_UNIT_ID,
            version=PARDOT_API_VERSION,
        )

        if not p.authenticate():
            p.campaigns.query(limit=1)
            # ^ Must call query to authenticate if authenticatation is unsuccessful

    except PardotAPIError as err:
        if err.message in [
            "Error #184: access_token is invalid, unknown, or malformed",
            "Error #122: Daily API rate limit met",
        ]:
            get_session_boto().client("sns").publish(
                TargetArn="arn:aws:sns:us-east-1:768217030320:test-topic-email-notification",
                Message=f"PARDOT ERROR: {err.message}",
                # Message=json.dumps({"default": json.dumps(message)}),
                MessageStructure="string",
            )
        raise

    assert (
        p.sftoken != "dummy"
    ), "Pardot authentication still unsuccessful after querying"

    return p


In [69]:
p = get_client_pardot()

In [11]:
date_start = dt.date.today() - dt.timedelta(days=100)

In [220]:
data_type = 'VisitorActivity'
key_mapping = {
    "VisitorActivity":"visitor_activity",
    "ListMembership":"list_membership",
    "Prospect":"prospect",
    "ProspectAccount":"prospectAccount",
    "Visitor":"visitor",
}
DICT_CONVERSION_PURAL = {
    "Account": "accounts",
    "EmailClick": "emailclicks",
    "Email": "emails",
    "VisitorActivity": "visitoractivities",
    "Campaign": "campaigns",
    "List": "lists",
    "Prospect": "prospects",
    "TagObject": "tagobjects",
    "Tag": "tags",
    "Opportunity": "opportunities",
    "ProspectAccounts": "prospectaccounts",
    "Form": "forms",
    "ListMembership": "listmemberships",
    "Visitor": "visitors",
    "ProspectAccount": "prospectaccounts",
}
data_client = getattr(p, DICT_CONVERSION_PURAL[data_type])

In [157]:
hasattr(data_client, "query")
data_client

In [9]:
def helper_to_camelCase(s: str) -> str:
    "Return the same string with the first letter lowercase"
    return str(s[0].lower() + s[1:])


In [237]:
"visitor_activity"


# # import datetime as dt
#dt.datetime.strptime(str("2021-12-31"), "%Y-%m-%d").date().isoformat()
# # # First pull the very first record
# dt.datetime.strptime(str("2020-01-01 23:59:59"), "%Y-%m-%d %H:%M:%S").isoformat()
# export_range = [["2021-01-01",dt.datetime.now().date()]]

# for date_range in export_range:
#     updated_after= dt.datetime.strptime(str(date_range[0]) + " 00:00:00", "%Y-%m-%d %H:%M:%S").isoformat()
#     updated_before= dt.datetime.strptime(str(date_range[1]) + " 23:59:59", "%Y-%m-%d %H:%M:%S").isoformat()
#     print(updated_after,updated_before)

data_raw = data_client.query(limit=200,updated_after="2021-01-01T00:00:00",
updated_before="2021-12-31T00:00:00")
# data_raw
# list_records = data_raw[helper_to_camelCase(data_type)]
# # list_records
# # early_date_str = early_record["visitor_activity"]["created_at"]
# # # early_date
# early_date_date = dt.datetime.strptime(early_date, "%Y-%m-%d %H:%M:%S").date()
# # Create list with every range of year's dates
# yr_range = [early_date_date.year,dt.datetime.now().year]
# export_range = [[f'01-01-{year}',f'12-31-{year}'] for year in range(yr_range[0],yr_range[1]+1)]


# now = dt.datetime.now().date()
# d = dt.timedelta(days = 365)

# if now - d > early_date_date:
#     print("Greater than one year")
#     print(now - d)
#     print(early_date_date)

# date_start,dt.datetime.now().date()
# data_raw = data_client.query(
#                 format="json",
#                 sort_by="created_at",
#                 sort_order="ascending",
                # limit=1)
#                 id_greater_than=0,
#                 updated_after=date_start.isoformat(),
#                 **(
#                     dict(created_after=date_start.isoformat())
#                     if data_type in SET_DATA_TYPE_CREATED
#                     else {}
#                 ),
#                  **(
#                     dict(type="Email")
#                     if data_type == "TagObject"
#                     else {}
#                 ),
                
#             )

# data_raw
# data_raw


2021-01-01T00:00:00 2022-02-01T23:59:59


In [167]:
def get_client_snowflake():
    return snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASS,
        account=SNOWFLAKE_ACCOUNT_IDENTIFIER,
        # authenticator="externalbrowser",
        warehouse="GENERAL_COMPUTE_WH",
        database="PROD_DATA_VAULT",
        schema="STAGE",
    )

In [280]:
ctx = get_client_snowflake()
cs = ctx.cursor()
name_table_snowflake = "PARDOT_OPPORTUNITY"
SF_SCHEMA = "PROD_DATA_VAULT"



try:
    cs.execute(f"select * from prod_data_vault.stage.PARDOT_OPPORTUNITY limit 10")
    data =  cs.fetchall()  
    columns = [f'{col[0]}' for col in cs.description]
finally:
    cs.close()
ctx.close()

pd.DataFrame(data,columns=columns)
# (1137682, 1137684)

,OPPORTUNITY_ID,OPPORTUNITY_NAME,VALUE,PROBABILITY,TYPE,STAGE,STATUS,CLOSED_AT,CREATED_AT,UPDATED_AT,CAMPAIGN,PROSPECTS,VAULTADDED,STAGEFILENAME
0,1949538,OPTY425590,4000.0,10,Renewal,Qualified,Open,NaT,2021-10-09 01:00:33-07:00,2022-01-28 09:53:36-08:00,"{'id': 12971, 'name': 'K12'}","{'prospect': [{'id': 40247804, 'first_name': '...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv
1,1949882,OPTY419589,4000.0,0,New,Closed Lost,Lost,2021-10-31 00:00:00-07:00,2021-07-13 12:05:16-07:00,2022-01-28 17:21:15-08:00,"{'id': 11286, 'name': 'Website Tracking'}","{'prospect': {'id': 50339838, 'first_name': 'H...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv
2,1949884,OPTY423736,5249.0,100,New,Closed Won,Won,2021-09-27 00:00:00-07:00,2021-09-24 12:51:08-07:00,2022-01-28 17:21:15-08:00,"{'id': 11286, 'name': 'Website Tracking'}","{'prospect': [{'id': 50339838, 'first_name': '...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv
3,1950584,OPTY419735,4448.0,100,New,Closed Won,Won,2021-09-02 00:00:00-07:00,2021-07-16 16:18:34-07:00,2022-01-27 16:35:09-08:00,"{'id': 11286, 'name': 'Website Tracking'}","{'prospect': {'id': 50352018, 'first_name': 'K...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv
4,1951506,OPTY425615,2009.0,100,New,Closed Won,Won,2021-10-13 00:00:00-07:00,2021-10-11 08:13:58-07:00,2022-01-29 10:26:29-08:00,"{'id': 20932, 'name': 'Marketing Cloud Migrati...","{'prospect': {'id': 21322488, 'first_name': 'J...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv
5,1951776,OPTY425620,11000.0,10,New,Qualified,Open,NaT,2021-10-11 10:13:57-07:00,2022-01-26 18:52:45-08:00,"{'id': 12967, 'name': 'Experiential'}","{'prospect': {'id': 18471623, 'first_name': 'T...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv
6,1952122,OPTY425627,0.0,0,Cross-Sell,Closed Lost,Lost,2021-12-31 00:00:00-08:00,2021-10-11 14:09:49-07:00,2022-01-28 07:32:22-08:00,"{'id': 12971, 'name': 'K12'}","{'prospect': {'id': 33309495, 'first_name': 'M...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv
7,1952262,OPTY425633,0.0,25,New,Value Prop/Proposal,Open,NaT,2021-10-11 15:29:31-07:00,2022-01-27 07:07:06-08:00,"{'id': 27647, 'name': 'mystery_science'}","{'prospect': {'id': 38061127, 'first_name': 'L...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv
8,1954013,OPTY425668,24453.0,10,New,Qualified,Open,NaT,2021-10-12 13:02:06-07:00,2022-01-27 15:56:19-08:00,"{'id': 12971, 'name': 'K12'}","{'prospect': {'id': 49999410, 'first_name': 'J...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv
9,1954429,SCC: Janus Henderson,1822000.0,50,New,Highly Qualified,Open,NaT,2021-10-12 16:27:09-07:00,2022-01-31 15:36:43-08:00,"{'id': 19077, 'name': 'Salesforce Generated'}","{'prospect': {'id': 28663499, 'first_name': 'D...",2022-02-01 14:49:34.443000-08:00,Opportunity/Opportunity_api_20220201-123456.csv


In [19]:
list_record = data_raw["tagObject"]
list_record

[{'id': 106036033,
  'tag_id': 33943,
  'type': 'Email',
  'object_id': 440415261,
  'created_at': '2021-10-23 13:32:40'},
 {'id': 106036035,
  'tag_id': 867471,
  'type': 'Email',
  'object_id': 440415261,
  'created_at': '2021-10-23 13:32:40'},
 {'id': 106036037,
  'tag_id': 1075496,
  'type': 'Email',
  'object_id': 440415261,
  'created_at': '2021-10-23 13:32:40'},
 {'id': 106036039,
  'tag_id': 33943,
  'type': 'Email',
  'object_id': 440415265,
  'created_at': '2021-10-23 11:59:57'},
 {'id': 106036041,
  'tag_id': 1075496,
  'type': 'Email',
  'object_id': 440415265,
  'created_at': '2021-10-23 11:59:58'},
 {'id': 106036043,
  'tag_id': 33943,
  'type': 'Email',
  'object_id': 440415267,
  'created_at': '2021-10-23 13:32:40'},
 {'id': 106036045,
  'tag_id': 867471,
  'type': 'Email',
  'object_id': 440415267,
  'created_at': '2021-10-23 13:32:40'},
 {'id': 106036047,
  'tag_id': 1075496,
  'type': 'Email',
  'object_id': 440415267,
  'created_at': '2021-10-23 13:32:40'},
 {'id': 

In [275]:
"Steve\n\n\n\n\n\n\n".count("\n")

7